In [1]:
# Path to the config file
config_path = "/Users/ryanarman/code/oumi/configs/examples/analyze/analyze.yaml"

In [2]:
import os
import sys


# IMPORTANT: Set these BEFORE importing torch or any ML libraries
# Disable all GPU/MPS backends to prevent crashes with IFD analyzer
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disable CUDA
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"  # Disable MPS memory allocation
os.environ["DISABLE_MPS_COMPAT"] = "1"  # Additional MPS disable flag
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"  # Disable HuggingFace telemetry
os.environ["TRANSFORMERS_OFFLINE"] = "0"  # Allow model downloads

# Force CPU usage in PyTorch to avoid MPS crashes
import torch

# Forcefully disable MPS before anything else
torch.set_default_device("cpu")
if hasattr(torch.backends, "mps"):
    # Monkey-patch to prevent MPS usage
    original_is_available = torch.backends.mps.is_available
    torch.backends.mps.is_available = lambda: False
    print("MPS has been disabled - forcing CPU-only mode")
else:
    print("Using CPU for all computations")

print(f"PyTorch device: {torch.get_default_device()}")
print(f"PyTorch version: {torch.__version__}")

MPS has been disabled - forcing CPU-only mode
PyTorch device: cpu
PyTorch version: 2.6.0


In [3]:
import os
from oumi.core.configs import AnalyzeConfig
from oumi.core.analyze.dataset_analyzer import DatasetAnalyzer

# Path to your dataset file
dataset_path = "/Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl"

# Load the config from YAML
config = AnalyzeConfig.from_yaml(
    config_path=config_path,
)

config.sample_count = 10
config.chat_template = "chat_ml"

# Override the dataset settings to use your local file
config.dataset_path = dataset_path
config.dataset_name = None  # Clear dataset_name so it uses dataset_path instead

# Optionally update output path
config.output_path = "./analysis_output/banking77"

# IMPORTANT: Disable analyzers that require large model downloads or have issues
# IFD requires downloading Qwen model and may cause MPS crashes
# fasttext requires additional dependencies
# repr_diversity and question_diversity download embedding models
print(
    f"Running {len(config.analyzers)} analyzers: {[a.instance_id for a in config.analyzers]}"
)

# config.analyzers = [a for a in config.analyzers if a.id == "length"]


# Validate the configuration
config.finalize_and_validate()

# Create the analyzer
analyzer = DatasetAnalyzer(config)

Running 18 analyzers: ['length', 'diversity', 'format', 'quality', 'training_quality', 'cost', 'content_pattern', 'question_diversity', 'fasttext', 'ifd', 'repr_diversity', 'task_category', 'safety', 'difficulty', 'instruct_reward', 'input_quality', 'conversation_structure', 'response_completeness']
[2026-01-05 16:54:20,402][oumi][rank0][pid:24381][MainThread][INFO]][base_map_dataset.py:91] Creating map dataset (type: TextSftJsonLinesDataset)... dataset_name: 'custom'
[2026-01-05 16:54:20,402][oumi.utils.analysis_utils][rank0][pid:24381][MainThread][INFO]][analysis_utils.py:227] Loaded text dataset from: /Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl
[2026-01-05 16:54:20,403][oumi][rank0][pid:24381][MainThread][INFO]][dataset_analyzer.py:154] Loaded dataset from config: None
[2026-01-05 16:54:20,526][oumi][rank0][pid:24381][MainThread][INFO]][dataset_analyzer.py:318] Initialized sample analyzer: length
[2026-01-05 16:54:20,526

In [4]:
# Run the analysis
analyzer.analyze_dataset()

# The results are stored in analyzer object
if analyzer._analysis_results:
    print(
        f"Total conversations analyzed: {analyzer._analysis_results.conversations_analyzed}"
    )

[2026-01-05 16:54:20,554][oumi][rank0][pid:24381][MainThread][INFO]][dataset_analyzer.py:361] Starting analysis of dataset: None
[2026-01-05 16:54:20,555][oumi][rank0][pid:24381][MainThread][INFO]][dataset_analyzer.py:362] Using 18 sample analyzers: ['length', 'diversity', 'format', 'quality', 'training_quality', 'cost', 'content_pattern', 'question_diversity', 'fasttext', 'ifd', 'repr_diversity', 'task_category', 'safety', 'difficulty', 'instruct_reward', 'input_quality', 'conversation_structure', 'response_completeness']
[2026-01-05 16:54:20,556][oumi][rank0][pid:24381][MainThread][INFO]][dataset_analyzer.py:385] Analyzing 10 of 8002 conversations
[2026-01-05 16:54:20,556][oumi][rank0][pid:24381][MainThread][INFO]][dataset_analyzer.py:435] Converting conversation dataset with 8002 items
[2026-01-05 16:54:20,556][oumi][rank0][pid:24381][MainThread][INFO]][dataset_analyzer.py:442] Limiting analysis to first 10 items (dataset has 8002 total)


Converting Unknown Dataset to DataFrames: 100%|██████████| 10/10 [00:00<00:00, 501.55item/s]


[2026-01-05 16:54:20,595][oumi.utils.analysis_utils][rank0][pid:24381][MainThread][INFO]][analysis_utils.py:1322] Adding default schema entries for 2 columns not in base schema: ['label', 'label_name']
[2026-01-05 16:54:20,625][oumi][rank0][pid:24381][MainThread][INFO]][question_diversity_analyzer.py:464] Computing embeddings for 10 user questions...
[2026-01-05 16:54:20,626][oumi][rank0][pid:24381][MainThread][INFO]][question_diversity_analyzer.py:174] Loading embedding model: all-MiniLM-L6-v2


Computing embeddings: 100%|██████████| 10/10 [00:00<00:00, 44.60it/s]


[2026-01-05 16:54:21,830][oumi][rank0][pid:24381][MainThread][INFO]][question_diversity_analyzer.py:469] Clustering 10 questions using dbscan...
[2026-01-05 16:54:22,237][oumi][rank0][pid:24381][MainThread][INFO]][question_diversity_analyzer.py:487] Found 1 clusters
[2026-01-05 16:54:22,259][oumi][rank0][pid:24381][MainThread][INFO]][fasttext_analyzer.py:220] Initialized fast-langdetect for language detection
[2026-01-05 16:54:22,260][oumi][rank0][pid:24381][MainThread][INFO]][fasttext_analyzer.py:436] Analyzing language for column: conversation_text_content
[2026-01-05 16:54:22,316][oumi][rank0][pid:24381][MainThread][INFO]][ifd_analyzer.py:154] Loading model for IFD analysis: Qwen/Qwen3-0.6B
[2026-01-05 16:54:24,169][oumi][rank0][pid:24381][MainThread][INFO]][ifd_analyzer.py:193] Loaded Qwen/Qwen3-0.6B on cpu (dtype: torch.float32)
[2026-01-05 16:54:24,170][oumi][rank0][pid:24381][MainThread][INFO]][ifd_analyzer.py:664] Computing IFD scores using instruction='conversation_text_conten

...

[2026-01-05 16:54:27,277][oumi][rank0][pid:24381][MainThread][INFO]][question_diversity_analyzer.py:469] Clustering 10 questions using dbscan...
[2026-01-05 16:54:27,280][oumi][rank0][pid:24381][MainThread][INFO]][question_diversity_analyzer.py:482] Found 0 clusters, 10 unique/diverse questions (not similar to others)
[2026-01-05 16:54:27,282][oumi][rank0][pid:24381][MainThread][INFO]][fasttext_analyzer.py:436] Analyzing language for column: text_content
[2026-01-05 16:54:27,288][oumi][rank0][pid:24381][MainThread][INFO]][ifd_analyzer.py:643] Detected conversation format. Computing IFD for assistant messages using preceding user messages as instructions.
[2026-01-05 16:54:29,419][oumi][rank0][pid:24381][MainThread][INFO]][ifd_analyzer.py:521] IFD analysis complete. Processed 10 assistant messages out of 10 total.
[2026-01-05 16:54:29,421][oumi][rank0][pid:24381][MainThread][INFO]][repr_diversity_analyzer.py:363] Computing diversity scores for 30 samples in column 'text_content'...


Computing embeddings: 100%|██████████| 30/30 [00:00<00:00, 160.41it/s]


[2026-01-05 16:54:29,610][oumi][rank0][pid:24381][MainThread][INFO]][repr_diversity_analyzer.py:230] Computing nearest neighbor distances for 30 samples (k=5)...
[2026-01-05 16:54:29,613][oumi][rank0][pid:24381][MainThread][INFO]][repr_diversity_analyzer.py:556] Column 'text_content': 19/30 samples (63.3%) are redundant


/Users/ryanarman/miniconda3/envs/oumi/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/ryanarman/miniconda3/envs/oumi/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


Total conversations analyzed: 10


In [5]:
schema = analyzer.get_schema()
print(f"len(schema): {len(schema)}")
merged_columns = analyzer.analysis_df.columns
print(f"len(merged_columns): {len(merged_columns)}")

len(schema): 186
len(merged_columns): 186


In [6]:
analyzer_names = [a.instance_id for a in config.analyzers]
analyzer_names

['length',
 'diversity',
 'format',
 'quality',
 'training_quality',
 'cost',
 'content_pattern',
 'question_diversity',
 'fasttext',
 'ifd',
 'repr_diversity',
 'task_category',
 'safety',
 'difficulty',
 'instruct_reward',
 'input_quality',
 'conversation_structure',
 'response_completeness']

# Conv level

In [7]:
analyzer_names

['length',
 'diversity',
 'format',
 'quality',
 'training_quality',
 'cost',
 'content_pattern',
 'question_diversity',
 'fasttext',
 'ifd',
 'repr_diversity',
 'task_category',
 'safety',
 'difficulty',
 'instruct_reward',
 'input_quality',
 'conversation_structure',
 'response_completeness']

In [8]:
analyzer.conversation_df.head()

,conversation_index,conversation_id,num_messages,conversation_text_content,label,label_name,conversation_text_content__length__token_count,conversation_text_content__diversity__unique_words_ratio,conversation_text_content__format__has_markdown,conversation_text_content__format__has_json,...,conversation_text_content__input_quality__tier,conversation_text_content__input_quality__score,conversation_text_content__input_quality__is_ambiguous,conversation_text_content__input_quality__is_answerable,conversation_text_content__input_quality__has_sufficient_context,conversation_text_content__response_completeness__is_complete,conversation_text_content__response_completeness__score,conversation_text_content__response_completeness__ends_naturally,conversation_text_content__response_completeness__has_conclusion,conversation_text_content__response_completeness__truncation_type
0,0,0,3,<|im_start|>system\nYou are a banking intent c...,52,request_refund,1806,0.516556,True,False,...,fair,0.42,True,True,True,False,0.4,False,False,incomplete_list
1,1,1,3,<|im_start|>system\nYou are a banking intent c...,69,verify_my_identity,1805,0.512707,True,False,...,good,0.66,False,True,True,False,0.4,False,False,incomplete_list
2,2,2,3,<|im_start|>system\nYou are a banking intent c...,59,top_up_failed,1801,0.518313,True,False,...,good,0.66,False,True,True,False,0.4,False,False,incomplete_list
3,3,3,3,<|im_start|>system\nYou are a banking intent c...,54,supported_cards_and_currencies,1807,0.513782,True,False,...,good,0.66,False,True,True,False,0.4,False,False,incomplete_list
4,4,4,3,<|im_start|>system\nYou are a banking intent c...,57,top_up_by_card_charge,1804,0.517127,True,False,...,good,0.66,False,True,True,False,0.4,False,False,incomplete_list


In [9]:
from oumi.core.analyze.column_utils import (
    filter_analyzer_columns,
    get_analyzer_columns_by_analyzer,
    parse_analyzer_column_name,
)

conv_columns = analyzer.conversation_df.columns
row = analyzer.conversation_df.iloc[0]

# Choose the analzyer to analyze
analyzer_name = analyzer_names[0]


filtered_cols = filter_analyzer_columns(conv_columns, analyzer_id=analyzer_name)
if filtered_cols:
    print(f"Analyzer: {analyzer_name}")
    info = parse_analyzer_column_name(filtered_cols[0])
    # print("\nInput:")
    # print(f"source_column: {info.source_column}")
    # print(f"{row[info.source_column]}\n")

    for col in filtered_cols:
        info = parse_analyzer_column_name(col)
        print(f"metric: {info.metric_name}")
        # print(f"type: {schema[col]['type']}")
        # print(f"content_type: {schema[col]['content_type']}")
        print(f"description: {schema[col]['description']}")
        print(f"value: {row[col]}")
        print("\n")
else:
    print(f"No columns found for analyzer: {analyzer_name}")


Analyzer: length
metric: token_count
description: Token count for conversation_text_content
value: 1806




# Message level

In [10]:
from oumi.core.analyze.column_utils import (
    filter_analyzer_columns,
    get_analyzer_columns_by_analyzer,
    parse_analyzer_column_name,
)

msg_columns = analyzer.message_df.columns
row = analyzer.message_df.iloc[1]

# Choose the analzyer to analyze
analyzer_name = analyzer_names[0]


filtered_cols = filter_analyzer_columns(msg_columns, analyzer_id=analyzer_name)
if filtered_cols:
    print(f"Analyzer: {analyzer_name}")
    info = parse_analyzer_column_name(filtered_cols[0])

    print("\nInput:")
    print(f"[{row['role']}]: {row[info.source_column]}\n")
    # print(f"source_column: {info.source_column}")
    print(f"{row[info.source_column]}\n")

    for col in filtered_cols:
        info = parse_analyzer_column_name(col)
        print(f"metric: {info.metric_name}")
        # print(f"type: {schema[col]['type']}")
        # print(f"content_type: {schema[col]['content_type']}")
        print(f"description: {schema[col]['description']}")
        print(f"value: {row[col]}")
        print("\n")
else:
    print(f"No columns found for analyzer: {analyzer_name}")


Analyzer: length

Input:
[user]: If I bought something I didn't like, can I get a refund?

If I bought something I didn't like, can I get a refund?

metric: token_count
description: Token count for text_content
value: 14


